In [33]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib as mpl

mpl.style.use("seaborn-v0_8-deep")
mpl.rcParams["figure.figsize"] = (14, 5)
mpl.rcParams["figure.dpi"] = 100

### Loading the datafile for cleaning and quick view

In [34]:
data = pd.read_csv('../data/raw/nigeria_telecom_transactions_messy_actual_cities.csv')

print(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55000 entries, 0 to 54999
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Transaction ID       49431 non-null  object 
 1   Customer ID          49515 non-null  object 
 2   Transaction Date     49445 non-null  object 
 3   Operator Name        55000 non-null  object 
 4   Transaction Type     55000 non-null  object 
 5   Transaction Amount   49566 non-null  object 
 6   Customer Age         49602 non-null  float64
 7   Customer Gender      49534 non-null  object 
 8   Customer Location    49584 non-null  object 
 9   Service Plan         55000 non-null  object 
 10  Data Usage (MB)      49533 non-null  object 
 11  Call Duration (min)  49587 non-null  object 
 12  SMS Sent             49467 non-null  float64
 13  Internet Package     55000 non-null  object 
 14  Transaction Status   55000 non-null  object 
dtypes: float64(2), object(13)
memory usa

,Transaction ID,Customer ID,Transaction Date,Operator Name,Transaction Type,Transaction Amount,Customer Age,Customer Gender,Customer Location,Service Plan,Data Usage (MB),Call Duration (min),SMS Sent,Internet Package,Transaction Status
0,66d1c0da-4597-4458-b4db-b2e94fe9c69f,f5754858-fe7d-4cc9-a1ff-80ce6d64ca90,NaN,Airtel,Ar1tel,NaN,27.0,NaN,Minna,Postpaid,4826.08,214.72,51.0,Daily,Pending
1,21ca8795-8ad9-47bc-a16b-e4654697ee52,NaN,2023-10-29T02:48:20.227058,9mobile,Bill Payment,4804.1,39.0,Other,Kaduna,Postpaid,3440,234.3,13.0,Weekly,Failed
2,ea6478bf-6a8c-4d91-bd63-b4464ebc5ae0,3052574b-ae63-4e67-8d86-5f8418ef6f66,2024-04-03T02:22:20.359918,9mobile,Bill Payment,19303.03,NaN,male,Kaduna,Prepaid,NaN,0.9,0.0,Weekly,Completed
3,NaN,15ad55e7-2c9e-4a84-9b9d-9e7d7747a177,2024-02-07T00:22:30.827156,9mobil3,Bill Payment,4094,NaN,Male,Owerri,Prepaid,2250.0,$16.38,20.0,Daily,Completed
4,a1165572-b0f4-42e2-9890-eb16ef0b8741,a4adc3b5-2b36-4ba5-b7d9-1a103bfb247a,2008-10-31 21:12:40,Glo,Data Purchase,NaN,36.0,female,Sokoto,Prepaid,3307.97,$92.61,58.0,Weekly,Failed


In [35]:
# Copying the data to a new dataframe so as to have what to compare the data to after cleaning
df = data.copy()

### Checking each Feature for errors in entry and fixing the datatype if need be

**Reusable Number of Missing Values Function**

In [70]:
def missing_values(column_name):
    """
    This function calculates and returns the number of missing values present in a specified column of the DataFrame.
    """
    return (f"Number of Missing values in the {column_name} column: {df[column_name].isnull().sum()}")

**Transaction Date Column**

In [71]:
# Checking the number of missing values
missing_values('Transaction Date')

'Number of Missing values in the Transaction Date column: 4956'

In [37]:
# Getting an understanding of the types of values in the column
df['Transaction Date'].sample(8)

12411    2024-04-20T12:25:42.562461
40291           1979-09-09 14:11:31
18014    2023-10-30T11:44:50.632640
48624           1983-09-24 20:40:14
29740    2024-07-18T22:11:09.882887
24301           2014-10-24 06:42:53
37505    2024-07-19T08:51:44.521008
39874           1983-04-26 23:21:15
Name: Transaction Date, dtype: object

In [38]:
# Converting the Transaction Date column to datetime and removing the nano-seconds from it
df['Transaction Date'] = (
                            pd.to_datetime
                            (
                                df['Transaction Date']
                                .str.replace('T', ' ', regex=False)
                                .str.split('.')
                                .str[0]
                                )
                        )


In [39]:
df['Transaction Date'].sample(5)

17612   2024-01-01 19:30:54
32507   2024-01-22 17:56:43
45562   1973-09-02 09:47:32
20296   2024-01-23 23:34:53
5059    2024-06-22 00:17:28
Name: Transaction Date, dtype: datetime64[ns]

In [40]:
# Confirming that the number of null values in 'Transaction Date' is equal to df['Transaction Date'] after convertion
num_nulls_original = data['Transaction Date'].isnull().sum()
num_nulls_modified = df['Transaction Date'].isnull().sum()
assert num_nulls_original == num_nulls_modified, "The number of null values in 'Transaction Date' must be the same before and after modifications."

# Confirming that all the values follow the same format
assert (df['Transaction Date'].dropna().dt.strftime('%Y-%m-%d %H:%M:%S') == df['Transaction Date'].dropna()).all(), "Not all non-NaT dates follow the '%Y-%m-%d %H:%M:%S' format"

**Operator Name Column**

- Checking amount of missing value

In [72]:
missing_values('Operator Name')

'Number of Missing values in the Operator Name column: 0'

- Getting the types of values in the column

In [42]:
# Getting the types of values in the column
df['Operator Name'].value_counts()

Operator Name
MTN        12560
Airtel     12340
9mobile    12322
Glo        12284
Ar1tel      1413
G1o         1397
9mobil3     1387
MNT         1297
Name: count, dtype: int64

- Correcting the spelling errors

In [43]:
correction_map = {
    'Ar1tel': 'Airtel',
    'G1o': 'Glo',
    '9mobil3': '9mobile',
    'MNT': 'MTN'
}

# Apply the corrections
df['Operator Name'] = df['Operator Name'].replace(correction_map)

In [44]:
df['Operator Name'].value_counts()

Operator Name
MTN        13857
Airtel     13753
9mobile    13709
Glo        13681
Name: count, dtype: int64

**Transaction Type Column**

In [73]:
missing_values('Transaction Type')

'Number of Missing values in the Transaction Type column: 0'

In [46]:
# Getting the types of values in the column
df['Transaction Type'].value_counts()

Transaction Type
Bill Payment        16548
Airtime Purchase    16536
Data Purchase       16455
9mobil3              1430
Ar1tel               1345
G1o                  1343
MNT                  1343
Name: count, dtype: int64

it seems there' an error in entry where operators names where included as collection type.
We would have to remove these rows(entries) since there's now way to know how to map them to actual transaction types

In [52]:
# Filtering out the rows with the wrong transaction type
df = df[~df['Transaction Type'].isin(['9mobil3', 'Ar1tel', 'G1o', 'MNT'])]
df['Transaction Type'] = df['Transaction Type'].astype('category')

**Transaction Amount**

In [67]:
missing_values('Transaction Amount')

'Number of Missing values in the Transaction Amount is: 4867'

<class 'pandas.core.frame.DataFrame'>
Index: 49539 entries, 1 to 54999
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Transaction ID       44530 non-null  object        
 1   Customer ID          44624 non-null  object        
 2   Transaction Date     44583 non-null  datetime64[ns]
 3   Operator Name        49539 non-null  object        
 4   Transaction Type     49539 non-null  category      
 5   Transaction Amount   44672 non-null  object        
 6   Customer Age         44705 non-null  float64       
 7   Customer Gender      44601 non-null  object        
 8   Customer Location    44719 non-null  object        
 9   Service Plan         49539 non-null  object        
 10  Data Usage (MB)      44589 non-null  object        
 11  Call Duration (min)  44671 non-null  object        
 12  SMS Sent             44576 non-null  float64       
 13  Internet Package     49539 non-null 